In [1]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

In [2]:
path = r'T:\DCProjects\StoryMap\BikeCounting\BikeMap\data\InteractiveBikeMap.gdb'
bike_friendly_businesses = gpd.read_file(path, layer="Bike_Friendly_Businesses")

In [3]:
bike_friendly_businesses.head()

,Status,Score,Match_type,Match_addr,Side,Ref_ID,User_fld,Addr_type,ARC_Street,ARC_City,ARC_State,ARC_ZIP,Account_Na,Street,City,Zip_Postal,Date_Appro,Applicant,Region,geometry
0,M,89.04,M,"91049 S WILLAMETTE ST, Coburg, OR, 97408",,98634,,PointAddress,91049 SOUTH WILLAMETTE; BOX 8542,COBURG,,,Reflections Salon,91049 South Willamette; Box 8542,Coburg,97408,2015-11-25T00:00:00,Vicki Bell,Willamette Valley,POINT (4247692.024 910514.051)
1,M,100.00,A,"91003 S WILLAMETTE ST, Coburg, OR, 97408",,101760,,PointAddress,91003 S Willamette St,Coburg,,,The Oregon Handwork Studio,91003 S Willamette St,Coburg,97408,2016-04-28T00:00:00,Kathy Paterson,Willamette Valley,POINT (4247890.889 910107.994)
2,M,100.00,A,"90999 S WILLAMETTE ST, Coburg, OR, 97408",,134476,,PointAddress,PO Box 8008 90999 S Willamette St,Coburg,,,Coburg Pizza Company,PO Box 8008 90999 S Willamette St,Coburg,97408,2014-11-20T00:00:00,Terry Dawson,Willamette Valley,POINT (4247953.708 909940.685)
3,M,100.00,M,"91136 N WILLAMETTE ST, Coburg, OR, 97408",,134510,,PointAddress,91136 N. WILLAMETTE STREET,COBURG,,,City of Coburg,PO Box 8316,Coburg,97408,2014-10-24T00:00:00,Petra Schuetz,Willamette Valley,POINT (4247265.300 911467.629)
4,U,0.00,A,,,-1,,,500 West 15th,Eugene,,,Fort Dalles Museum,500 West 15th,Eugene,97058,2014-05-12T00:00:00,Trish Neal,Mt. Hood/Gorge,POINT (6052081273769120.000 6052081291735920.000)


In [4]:
bike_friendly_businesses.columns

Index(['Status', 'Score', 'Match_type', 'Match_addr', 'Side', 'Ref_ID',
       'User_fld', 'Addr_type', 'ARC_Street', 'ARC_City', 'ARC_State',
       'ARC_ZIP', 'Account_Na', 'Street', 'City', 'Zip_Postal', 'Date_Appro',
       'Applicant', 'Region', 'geometry'],
      dtype='object')